# Task 3: Baseline Model Comparison (Neural Network)

In this notebook, you will train a simple Neural Network (MLP) to predict the `prior_hiring_decision` target variable. Compare its performance to the previous models.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

# Load Data
train_df = pd.read_csv('../data/train.csv')
val_df = pd.read_csv('../data/val.csv')
test_df = pd.read_csv('../data/test.csv')

print("Training Shape:", train_df.shape)

## Model Training & Evaluation
**TODO:** Implement your Neural Network (MLPClassifier) pipeline here. Pay attention to data scaling.

In [ ]:
# TODO: Train Neural Network and Report Accuracy + Fairness Metrics